# Hamiltonian Monte Carlo (distinguished from Dian Zhang's Model)

#### Theory
With reference to [Stan Reference Manual](https://mc-stan.org/docs/reference-manual/hamiltonian-monte-carlo.html).



- The goal is to draw samples from a target distribution $p(\theta)$ for parameters $\theta$. (Dian uses $\rho(\theta)$ for the probability distribution)

- HMC introduces auxilary momentum variables $\rho$, this auxilary density $\rho$ is a multivariate normal and is independent of $\theta$.

    - $\rho \sim MultiNormal(\underline{0}, M)$, where $M$ is the [Euclidean metric](https://mathworld.wolfram.com/EuclideanMetric.html).

        - eg. a 2D Gaussian $p(\textbf{x}) = \mathcal{N}(\textbf{x};\begin{bmatrix} 0 \\ 0 \end{bmatrix},\begin{bmatrix} 1 & 0.98 \\ 0.98 & 1 \end{bmatrix})$

- The Hamiltonian $H(\rho,\theta)= kinetic \ energy + potential \ energy$

    - $kinetic \ energy = T (\rho \mid \theta) = - \log p(\rho \mid \theta)$

    - $potential \ energy = U(\theta) = -\log p(\theta)$

    - $\frac{d\theta}{dt} = -\frac{\partial T}{\partial \rho}$
    
    - $\frac{d\rho}{dt} = -\frac{\partial U}{\partial \theta}$

**Leapfrog integrator**

For small time inteval $\epsilon$, it updates the $\rho$ and $\theta$ as follows:
- $$\rho \leftarrow \rho - \frac{\epsilon}{2} \frac{\partial U}{\partial \theta}$$
- $$\theta \leftarrow \theta + \epsilon M^{-1} \rho$$
- $$\rho \leftarrow \rho - \frac{\epsilon}{2} \frac{\partial U}{\partial \theta}$$

After the orbit is integrated for a while, a new proposed sample is generated, and accepted or rejected,
then a new random momentum is generated and the procedure repeated.

## Parameters

With reference to [Stan Reference Manual](https://mc-stan.org/docs/reference-manual/hmc-algorithm-parameters.html)

The Hamiltonian Monte Carlo algorithm has three parameters which must be set,

- discretization time $\epsilon$
- metric $M$
- number of leapfrog steps taken $L$

In [ ]:
import numpy as np

In [1]:
# # Define a function to calculate the Hamiltonian
# def hamiltonian(x, p):
#     return 0.5 * np.dot(p, p) + np.sum(np.square(x))

# # Define a function to calculate the partial derivatives
# def hamiltonian_derivatives(x, p):
#     return np.array([2*x, p])

# # Define a function to calculate the Hamiltonian Monte Carlo updates
# def hmc_updates(x, p, epsilon, L):
#     current_hamiltonian = hamiltonian(x, p)
#     # Initialize the position and momentum
#     x_new = x
#     p_new = p
#     # Perform the leapfrog updates
#     for i in range(L):
#         p_new = p_new - epsilon * hamiltonian_derivatives(x_new, p_new)[0] / 2
#         x_new = x_new + epsilon * hamiltonian_derivatives(x_new, p_new)[1]
#         p_new = p_new - epsilon * hamiltonian_derivatives(x_new, p_new)[0] / 2
#     # Calculate the new Hamilitonian
#     new_hamiltonian = hamiltonian(x_new, p_new)
#     # Accept or reject the new position based on the Metropolis-Hastings criterion
#     if np.random.uniform(0, 1) < np.exp(current_hamiltonian - new_hamiltonian):
#         return x_new
#     else:
#         return x

# # Define a function to run the Hamiltonian Monte Carlo
# def hmc(x_init, epsilon, L, n_steps):
#     x = x_init
#     p = np.random.normal(size=x_init.shape)
#     for i in range(n_steps):
#         x = hmc_updates(x, p, epsilon, L)
#     return x

In [ ]:
def hamiltonian_monte_carlo(epoch, L, epsilon, Cov):
    """
    
    Hamiltonian Monte Carlo algorithm for a bivariate normal

    Parameters
    ----------
    epoch: number of iteration of the algorithm
    Cov: covariance matrix
    L: number of steps of leap frog
    epsilon: step size for discrete approximation
    
    Returns
    -------
    theta : ndarray of shape 2 x N. Transformed variable with Hamiltonian samples

    
    """
    
    def potential_energy(x, Cov):
        return x.dot(np.linalg.inv(Cov.T)).dot(x.T)

    def grad_potential_energy(x, Cov):
        return x.dot(np.linalg.inv(Cov))

    def kinetic_energy(p):
        return np.sum(p.T.dot(p)/2)
    
    theta = np.zeros(2*epoch).reshape(epoch, 2)

    for t in range(1, epoch):

        #   SAMPLE RANDOM MOMENTUM
        rho0 = np.random.randn(2)

        #   SIMULATE HAMILTONIAN DYNAMICS
        #   FIRST 1/2 STEP OF MOMENTUM
        rho_star = rho0 - 1/2*epsilon*grad_potential_energy(theta[t-1,:], Cov)

        #   FIRST FULL STEP FOR POSITION/SAMPLE
        theta_star = theta[t-1,:] + epsilon*rho_star

        #   FULL STEPS
        for i in range(0, L):
            # MOMENTUM
            rho_star = rho_star - epsilon*grad_potential_energy(theta_star, Cov)
            # POSITION/SAMPLE
            theta_star = theta_star + epsilon*rho_star

        rho_star = rho_star - 1/2*epsilon*grad_potential_energy(theta_star, Cov)

        U0 = potential_energy(theta[t-1,:], Cov)
        U_star = potential_energy(theta_star, Cov)

        K0 = kinetic_energy(rho0)
        K_star = kinetic_energy(rho_star)

        # acceptance ratio
        alpha = min(1, np.exp((U0 + K0) - (U_star + K_star)))

        # accept/reject criterion        
        theta[t,:] = theta_star if np.random.random() < alpha else theta[t-1,:]
        
    return theta